In [5]:
import os
import h5py
import json
import numpy as np
import sys
import glob
import typing
import copy
from typing import List, Dict, Union
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from collections import Counter, OrderedDict, defaultdict
# % matplotlib inline
sys.path.append("../") # Append visdialch path

In [2]:
path_train_data = "../../data/visdial_1.0_train.json"
dense_annotations_jsonpath = "../../data/visdial_1.0_train_dense_annotations.json"

In [28]:
def convert_list_json_dic(ranks_json):
    image_ranks_dic = {}
    for i in range(len(ranks_json)):
        image_ranks_dic[ranks_json[i]["image_id"]] = ranks_json[i]
    return image_ranks_dic

unique_dense_opts = defaultdict()
unique_dense_opts_with_relevance = defaultdict()

In [29]:
data_train = json.load(open(path_train_data))
annotations_json = json.load(open(dense_annotations_jsonpath))

questions = data_train['data']['questions']
len(questions)

answers = data_train['data']['answers']
len(answers)

dialogs = data_train['data']['dialogs']

In [30]:
dialogs_dic = convert_list_json_dic(dialogs)

In [31]:
print(len(dialogs))
print(len(dialogs_dic.keys()))
img_ids = list(dialogs_dic.keys())
print(img_ids[0])

# dialogs_dic[378466]

123287
123287
378466


{'image_id': 378466,
 'dialog': [{'answer': 52236,
   'gt_index': 74,
   'question': 73533,
   'answer_options': [94932,
    285447,
    29881,
    312030,
    273807,
    34555,
    314677,
    293546,
    53334,
    240597,
    104247,
    69709,
    149536,
    11790,
    79597,
    98241,
    120539,
    206767,
    163448,
    104516,
    54352,
    267512,
    143078,
    323883,
    297952,
    272640,
    56362,
    141689,
    99249,
    45558,
    120165,
    20691,
    102363,
    295401,
    10768,
    266800,
    21932,
    55828,
    38730,
    277082,
    290089,
    314061,
    263410,
    199957,
    229194,
    139398,
    89917,
    47436,
    116867,
    222919,
    333994,
    68998,
    188963,
    121123,
    170034,
    126639,
    181099,
    319289,
    316444,
    312182,
    53649,
    189677,
    187250,
    209519,
    29153,
    205885,
    51111,
    236568,
    69429,
    271915,
    331114,
    7690,
    295090,
    32702,
    52236,
    63063,
    173

In [32]:
for ann_indx in range(len(annotations_json)):

    annotation_line = annotations_json[ann_indx]
    image_id = annotation_line["image_id"]
    round_id = annotation_line["round_id"]  # 1-index
    relevance = annotation_line["relevance"]
    # Get option list from dialog json
    dial_for_image = dialogs_dic[image_id]
    answer_options_list = dial_for_image['dialog'][round_id-1]['answer_options'] # 0-index    
    for opn_indx in range(len(relevance)):
        # We add only options with non-zero relevance
        opt_rel = relevance[opn_indx]
        if relevance[opn_indx] != 0:
            ans_key = answers[answer_options_list[opn_indx]] 
            unique_dense_opts[ans_key] = unique_dense_opts.get(ans_key, 0) + 1            
            if ans_key in unique_dense_opts_with_relevance:
                unique_dense_opts_with_relevance[ans_key].append(opt_rel)
            else:
                unique_dense_opts_with_relevance[ans_key] = [opt_rel]

In [37]:
import math
import statistics

In [55]:
unique_dense_opts_with_avg_relevance = defaultdict()
for key, val in unique_dense_opts_with_relevance.items():
    avg_val = statistics.mean(val)
    if len(val) > 20:
        unique_dense_opts_with_avg_relevance[key] = avg_val        

#     if avg_val < 0.8:
#         unique_dense_opts_with_avg_relevance[key] = avg_val

In [56]:
unique_dense_opts_with_avg_relevance = Counter(unique_dense_opts_with_avg_relevance)
most_common_opts = OrderedDict(unique_dense_opts_with_avg_relevance.most_common(20))

print(len(unique_dense_opts))
for item in most_common_opts.items():
    print(item[0], ":", item[1])

3652
yes, it is a color photo : 0.9918032786885246
yes it is in color : 0.9791666666666666
yes color picture : 0.9565217391304348
yes i can : 0.9102564102564102
yes, it is : 0.9081632653061225
no i can't : 0.8846153846153846
no i don't : 0.8809523809523809
no there is not : 0.8787878787878788
yes there is : 0.875
no i cannot : 0.875
yes, : 0.84375
yes : 0.8418740849194729
no : 0.8400286944045912
no people : 0.8305084745762712
nope : 0.82992700729927
yep : 0.8285714285714286
no it is not : 0.8269230769230769
brown : 0.8243243243243243
yes! : 0.8214285714285714
not at all : 0.8055555555555556


In [9]:
# ques_df = pd.DataFrame.from_dict(unique_ques, orient='index')

In [14]:
unique_dense_opts = Counter(unique_dense_opts)
most_common_opts = OrderedDict(unique_dense_opts.most_common(20))

print(len(unique_dense_opts))
for item in most_common_opts.items():
    print(item[0], ":", item[1])

3652
no -- 697
nope -- 685
yes -- 683
not that i can see -- 625
i think so -- 460
i can't tell -- 457
can't tell -- 445
not sure -- 427
yes it is -- 362
not really -- 342
maybe -- 212
0 -- 193
yes, it is -- 147
i don't think so -- 96
appears to be -- 92
looks like it -- 91
not visible -- 73
white -- 71
i don't see any -- 69
it is -- 63
